## Script for conducting IS-RSA analysis on emotion data

### Basic settings

In [ ]:
%matplotlib inline
import os
import mne
import pandas as pd
import numpy as np
from neurora.stuff import permutation_corr
from mne.viz import plot_topomap
from mne.stats import fdr_correction
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
jtplot.style(theme='grade3') 
from Function import spearmanr, permutation_cor

# Define the path
eeg_path = 'F:/1_Emotion_Data/Results/0_ISC/EEG/'
data_path = 'F:/1_Emotion_Data/Results/0_ISC/ISC_CSV/'
behav_path = 'F:/1_Emotion_Data/Results/0_ISC/Behav/'
results_path = 'F:/1_Emotion_Data/Results/1_IS_RSA/'
figure_path = 'F:/1_Emotion_Data/Results/2_Plots/ISRSA/4_edition/'

In [ ]:
# Use the mne.read_epochs_eeglab() to read the preprocessed data
demo = mne.read_epochs_eeglab('F:/1_Emotion_Data/Data/EEG/Angry/ag1/sub_001_ag1.set')
montage = mne.channels.read_custom_montage('F:/1_Emotion_Data/Data/EEG/Emotion.loc')
demo.set_montage(montage)

# Obtain channel & montage information for topo-plot
topo_info = demo.info
# Obtain the channel names as a list
ch_names = demo.ch_names
# Crate a index list for channels
ch_idx = list(range(63))
# Combine the channels and index and convert to a dict
ch = dict(zip(ch_names, ch_idx))
print(ch['TP8'])

### Import all ISC upper traingle matrics

In [ ]:
behav_simi = np.load(os.path.join(behav_path, 'behavisc_annak.npy'), allow_pickle=True).item()

ag_psd = np.load(os.path.join(eeg_path, 'ag_isc_matrix.npy'), allow_pickle=True).item()['matrix']
ag_physio = pd.read_csv(os.path.join(data_path, 'Physiology/ag_physio_isc.csv'))

ax_psd = np.load(os.path.join(eeg_path, 'ax_isc_matrix.npy'), allow_pickle=True).item()['matrix']
ax_physio = pd.read_csv(os.path.join(data_path, 'Physiology/ax_physio_isc.csv'))

fe_psd = np.load(os.path.join(eeg_path, 'fe_isc_matrix.npy'), allow_pickle=True).item()['matrix']
fe_physio = pd.read_csv(os.path.join(data_path, 'Physiology/fe_physio_isc.csv'))

hl_psd = np.load(os.path.join(eeg_path, 'hl_isc_matrix.npy'), allow_pickle=True).item()['matrix']
hl_physio = pd.read_csv(os.path.join(data_path, 'Physiology/hl_physio_isc.csv'))

ha_psd = np.load(os.path.join(eeg_path, 'ha_isc_matrix.npy'), allow_pickle=True).item()['matrix']
ha_physio = pd.read_csv(os.path.join(data_path, 'Physiology/ha_physio_isc.csv'))

In [ ]:
def get_fdr(isrsa_matrix):  
    _,delta_cor = fdr_correction(isrsa_matrix['delta_p'], alpha=0.05, method='indep')
    _,theta_cor = fdr_correction(isrsa_matrix['theta_p'], alpha=0.05, method='indep')
    _,alpha_cor = fdr_correction(isrsa_matrix['alpha_p'], alpha=0.05, method='indep')
    _,beta_cor = fdr_correction(isrsa_matrix['beta_p'], alpha=0.05, method='indep')
    _,gamma_cor = fdr_correction(isrsa_matrix['gamma_p'], alpha=0.05, method='indep')
    isrsa_matrix.insert(4, 'delta_p_cor', delta_cor)
    isrsa_matrix.insert(7, 'theta_p_cor', theta_cor)
    isrsa_matrix.insert(10, 'alpha_p_cor', alpha_cor)
    isrsa_matrix.insert(13, 'beta_p_cor', beta_cor)
    isrsa_matrix.insert(16, 'gamma_p_cor', gamma_cor)
    return isrsa_matrix

In [ ]:
def isrsa(psd_simi, behav_simi,ch_names):
    delta_r,delta_p, theta_r, theta_p, alpha_r, alpha_p, beta_r, beta_p, gamma_r, gamma_p  = [], [], [], [], [], [], [], [], [], []

    for ch in ch_idx:
        r = spearmanr(psd_simi[:,ch,0], behav_simi)
        p = permutation_corr(psd_simi[:,ch,0], behav_simi, method='spearman', iter=10000)
        delta_r.append(r)
        delta_p.append(p)
        
        r = spearmanr(psd_simi[:,ch,1], behav_simi)
        p = permutation_corr(psd_simi[:,ch,1], behav_simi, method='spearman', iter=10000)
        theta_r.append(r)
        theta_p.append(p)
        
        r = spearmanr(psd_simi[:,ch,2], behav_simi)
        p = permutation_corr(psd_simi[:,ch,2], behav_simi, method='spearman', iter=10000)
        alpha_r.append(r)
        alpha_p.append(p)
        
        r = spearmanr(psd_simi[:,ch,3], behav_simi)
        p = permutation_corr(psd_simi[:,ch,3], behav_simi, method='spearman', iter=10000)
        beta_r.append(r)
        beta_p.append(p)
        
        r = spearmanr(psd_simi[:,ch,4], behav_simi)
        p = permutation_corr(psd_simi[:,ch,4], behav_simi, method='spearman', iter=10000)
        gamma_r.append(r)
        gamma_p.append(p)

    isrsa_df = {'channel_name': ch_names, 'delta_r':delta_r, 'delta_p':delta_p, 'theta_r':theta_r, 'theta_p':theta_p, 'alpha_r':alpha_r, \
        'alpha_p':alpha_p, 'beta_r': beta_r, 'beta_p': beta_p, 'gamma_r': gamma_r, 'gamma_p': gamma_p}
    isrsa_df = pd.DataFrame(isrsa_df, dtype='float64')

    return isrsa_df


In [ ]:
def isrsa_topo(isrsa_matrix,topo_info,vmin,vmax,title):
    fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))

    mask = obtain_mask(isrsa_matrix['delta_p'], ch_idx)
    im,_ = plot_topomap(isrsa_matrix['delta_r'], topo_info, axes=ax1, show=False, mask=mask, mask_params=dict(marker='o',markersize=6, markerfacecolor='w'), vmin=vmin, vmax=vmax,cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['theta_p'], ch_idx)
    im,_ = plot_topomap(isrsa_matrix['theta_r'], topo_info, axes=ax2, show=False, mask=mask, mask_params=dict(marker='o',markersize=6, markerfacecolor='w'), vmin=vmin, vmax=vmax,  cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['alpha_p'], ch_idx)    
    im,_ = plot_topomap(isrsa_matrix['alpha_r'], topo_info, axes=ax3, show=False, mask=mask, mask_params=dict(marker='o',markersize=6, markerfacecolor='w'), vmin=vmin, vmax=vmax,  cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['beta_p'], ch_idx)    
    im,_ = plot_topomap(isrsa_matrix['beta_r'], topo_info, axes=ax4, show=False, mask=mask, mask_params=dict(marker='o',markersize=6, markerfacecolor='w'), vmin=vmin, vmax=vmax, cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['gamma_p'], ch_idx)    
    im,_ = plot_topomap(isrsa_matrix['gamma_r'], topo_info, axes=ax5, show=False, mask=mask, mask_params=dict(marker='o',markersize=6, markerfacecolor='w'), vmin=vmin, vmax=vmax, cmap='RdBu_r',sphere=0.13)

    ax1.set_title('Delta',fontsize=20) 
    ax2.set_title('Theta',fontsize=20) 
    ax3.set_title('Alpha',fontsize=20) 
    ax4.set_title('Beta',fontsize=20) 
    ax5.set_title('Gamma',fontsize=20) 
    ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.02, 0.4, 0.3
    cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
    clb = fig.colorbar(im, cax=cbar_ax)
    fig.suptitle(title, x=0.5, y=0.75, fontsize=25, fontweight='bold')
    plt.show()
    # plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISRSA/ag_so.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
    # plt.close()
    return fig
        

In [ ]:
# Obtain mask
def obtain_mask(p_vector, ch_idx):  
    tp = np.array(p_vector)
    for i in ch_idx: 
        p = tp[i]
        if p <= 0.05:
            tp[i] = True
        else:
            tp[i] = False
    return tp

### Section 1: IS-RSA of EEG PSD & Scales

### 1. Angry

#### Angey PSD & IMQ-SS

In [ ]:
ag_ss = isrsa(ag_psd, behav_simi['ag_ss'], ch_names)
ag_ss.to_csv(os.path.join(results_path, '1_side/ag_ss_isrsa.csv'))
isrsa_topo(ag_ss,topo_info,vmin=-0.15,vmax=0.15,title='Angry & IMQ-SS')

In [ ]:
ag_ss_cor = get_fdr(ag_ss)
ag_ss_cor.to_csv(os.path.join(results_path, '1_side/ag_ss_isrsa_cor.csv'))

#### Angry PSD & IMQ-SO

In [ ]:
ag_so = isrsa(ag_psd, behav_simi['ag_so'], ch_names)
ag_so.to_csv(os.path.join(results_path, '1_side/ag_so_isrsa.csv'))

In [ ]:
ag_so_cor = get_fdr(ag_so)
ag_so_cor.to_csv(os.path.join(results_path, '1_side/ag_so_isrsa_cor.csv'))

#### Angry PSD & IRI-EC

In [ ]:
ag_ec = isrsa(ag_psd, behav_simi['ag_ec'], ch_names)
ag_ec.to_csv(os.path.join(results_path, '1_side/ag_ec_isrsa.csv'))

In [ ]:
ag_ec_cor = get_fdr(ag_ec)
ag_ec_cor.to_csv(os.path.join(results_path, '1_side/ag_ec_isrsa_cor.csv'))

#### Angry PSD & IRI-PD

In [ ]:
ag_pd = isrsa(ag_psd, behav_simi['ag_pd'], ch_names)
ag_pd.to_csv(os.path.join(results_path, '1_side/ag_pd_isrsa.csv'))
isrsa_topo(ag_pd,topo_info,vmin=-0.15, vmax=0.15, title='Angry & IRI-PD')

In [ ]:
ag_pd_cor = get_fdr(ag_pd)
ag_pd_cor.to_csv(os.path.join(results_path, '1_side/ag_pd_isrsa_cor.csv'))

### 2. Anxiety

#### Anxiety PSD & IMQ-SS

In [ ]:
ax_ss = isrsa(ax_psd, behav_simi['ax_ss'], ch_names)
ax_ss.to_csv(os.path.join(results_path, '1_side/ax_ss_isrsa.csv'))
isrsa_topo(ax_ss,topo_info,vmin=-0.15,vmax=0.15, title='Anxiety & IMQ-SS')

In [ ]:
ax_ss_cor = get_fdr(ax_ss)
ax_ss_cor.to_csv(os.path.join(results_path, '1_side/ax_ss_isrsa_cor.csv'))

#### Anxiety PSD & IMQ-SO

In [ ]:
ax_so = isrsa(ax_psd, behav_simi['ax_so'], ch_names)
ax_so.to_csv(os.path.join(results_path, '1_side/ax_so_isrsa.csv'))

In [ ]:
ax_so_cor = get_fdr(ax_so)
ax_so_cor.to_csv(os.path.join(results_path, '1_side/ax_so_isrsa_cor.csv'))

#### Anxiety PSD & IRI-EC

In [ ]:
ax_ec = isrsa(ax_psd, behav_simi['ax_ec'], ch_names)
ax_ec.to_csv(os.path.join(results_path, '1_side/ax_ec_isrsa.csv'))
isrsa_topo(ax_ec,topo_info,vmin=-0.15,vmax=0.15, title='Anxiety & IRI-EC')

In [ ]:
ax_ec_cor = get_fdr(ax_ec)
ax_ec_cor.to_csv(os.path.join(results_path, '1_side/ax_ec_isrsa_cor.csv'))

#### Anxiety PSD & IRI-PD

In [ ]:
ax_pd = isrsa(ax_psd, behav_simi['ax_pd'], ch_names)
ax_pd.to_csv(os.path.join(results_path, '1_side/ax_pd_isrsa.csv'))
isrsa_topo(ax_pd,topo_info,vmin=-0.15,vmax=0.15, title='Anxiety & IRI-PD')

In [ ]:
ax_pd_cor = get_fdr(ax_pd)
ax_pd_cor.to_csv(os.path.join(results_path, '1_side/ax_pd_isrsa_cor.csv'))

### 3. Fear

#### Fear PSD & IMQ-SS

In [ ]:
fe_ss = isrsa(fe_psd, behav_simi['fe_ss'], ch_names)
fe_ss.to_csv(os.path.join(results_path, '1_side/fe_ss_isrsa.csv'))
isrsa_topo(fe_ss,topo_info,vmin=-0.15,vmax=0.15,title='Fear & IMQ-SS')

In [ ]:
fe_ss_cor = get_fdr(fe_ss)
fe_ss_cor.to_csv(os.path.join(results_path, '1_side/fe_ss_isrsa_cor.csv'))

#### Fear PSD & IMQ-SO

In [ ]:
fe_so = isrsa(fe_psd, behav_simi['fe_so'], ch_names)
fe_so.to_csv(os.path.join(results_path, '1_side/fe_so_isrsa.csv'))

In [ ]:
fe_so_cor = get_fdr(fe_so)
fe_so_cor.to_csv(os.path.join(results_path, '1_side/fe_so_isrsa_cor.csv'))

#### Fear PSD & IRI-EC

In [ ]:
fe_ec = isrsa(fe_psd, behav_simi['fe_ec'], ch_names)
fe_ec.to_csv(os.path.join(results_path, '1_side/fe_ec_isrsa.csv'))

In [ ]:
fe_ec_cor = get_fdr(fe_ec)
fe_ec_cor.to_csv(os.path.join(results_path, '1_side/fe_ec_isrsa_cor.csv'))

#### Fear PSD & IRI-PD

In [ ]:
fe_pd = isrsa(fe_psd, behav_simi['fe_pd'], ch_names)
fe_pd.to_csv(os.path.join(results_path, '1_side/fe_pd_isrsa.csv'))
isrsa_topo(fe_pd,topo_info,vmin=-0.15,vmax=0.15, title='Fear & IRI-PD')

In [ ]:
fe_pd_cor = get_fdr(fe_pd)
fe_pd_cor.to_csv(os.path.join(results_path, '1_side/fe_pd_isrsa_cor.csv'))

### 4. Helpless

#### Helpless PSD & IMQ-SS

In [ ]:
hl_ss = isrsa(hl_psd, behav_simi['hl_ss'], ch_names)
hl_ss.to_csv(os.path.join(results_path, '1_side/hl_ss_isrsa.csv'))
isrsa_topo(hl_ss,topo_info,vmin=-0.15, vmax=0.15, title='Helpless & IMQ-SS')

In [ ]:
hl_ss_cor = get_fdr(hl_ss)
hl_ss_cor.to_csv(os.path.join(results_path, '1_side/hl_ss_isrsa_cor.csv'))

#### Helpless PSD & IMQ-SO

In [ ]:
hl_so = isrsa(hl_psd, behav_simi['hl_so'], ch_names)
hl_so.to_csv(os.path.join(results_path, '1_side/hl_so_isrsa.csv'))

In [ ]:
hl_so_cor = get_fdr(hl_so)
hl_so_cor.to_csv(os.path.join(results_path, '1_side/hl_so_isrsa_cor.csv'))

#### Helpless PSD & IRI-EC

In [ ]:
hl_ec = isrsa(hl_psd, behav_simi['hl_ec'], ch_names)
hl_ec.to_csv(os.path.join(results_path, '1_side/hl_ec_isrsa.csv'))

In [ ]:
hl_ec_cor = get_fdr(hl_ec)
hl_ec_cor.to_csv(os.path.join(results_path, '1_side/hl_ec_isrsa_cor.csv'))

#### Helpless PSD & IRI-PD

In [ ]:
hl_pd = isrsa(hl_psd, behav_simi['hl_pd'], ch_names)
hl_pd.to_csv(os.path.join(results_path, '1_side/hl_pd_isrsa.csv'))
isrsa_topo(hl_pd,topo_info,vmin=-0.15,vmax=0.15, title='Helpless & IRI-PD')

In [ ]:
hl_pd_cor = get_fdr(hl_pd)
hl_pd_cor.to_csv(os.path.join(results_path, '1_side/hl_pd_isrsa_cor.csv'))

### 5. Happy

#### Happy PSD & IMQ-SS

In [ ]:
ha_ss = isrsa(ha_psd, behav_simi['ha_ss'], ch_names)
ha_ss.to_csv(os.path.join(results_path, '1_side/ha_ss_isrsa.csv'))
isrsa_topo(ha_ss,topo_info,vmin=-0.15,vmax=0.15, title='Happy & IMQ-SS')

In [ ]:
ha_ss_cor = get_fdr(ha_ss)
ha_ss_cor.to_csv(os.path.join(results_path, '1_side/ha_ss_isrsa_cor.csv'))

#### Happy PSD & IMQ-SO

In [ ]:
ha_so = isrsa(ha_psd, behav_simi['ha_so'], ch_names)
ha_so.to_csv(os.path.join(results_path, '1_side/ha_so_isrsa.csv'))

In [ ]:
ha_so_cor = get_fdr(ha_so)
ha_so_cor.to_csv(os.path.join(results_path, '1_side/ha_so_isrsa_cor.csv'))

#### Happy PSD & IRI-FS

In [ ]:
ha_fs = isrsa(ha_psd, behav_simi['ha_fs'], ch_names)
ha_fs.to_csv(os.path.join(results_path, '1_side/ha_fs_isrsa.csv'))
isrsa_topo(ha_fs,topo_info,vmin=-0.15, vmax=0.15, title='Happy & IRI-FS')

In [ ]:
ha_fs_cor = get_fdr(ha_fs)
ha_fs_cor.to_csv(os.path.join(results_path, '1_side/ha_fs_isrsa_cor.csv'))

### Section 2: Psychophysiological signal & Scale

In [ ]:
ag_physio_scale = {}
for i in ('HR', 'EDR', 'HRV_MeanNN', 'HRV_SDNN'):
    for j in ('ag_ss', 'ag_so', 'ag_ec', 'ag_pd'):
        m1, m2 = ag_physio[i], behav_simi[j]
        r = spearmanr(m1, m2)
        p = permutation_corr(m1,m2,method='spearman', iter=10000)
        tp = 'Angry ' + i +  ' & ' + j
        ag_physio_scale[tp] = r
        if p <= 0.05:
            print('Angry ' + i +  ' & ' + j + ' correlation=' + str(r) + ' p=' + str(p))
        else:
            pass

ag_physio_scale

In [ ]:
ax_physio_scale = {}
for i in ('HR', 'EDR', 'HRV_MeanNN', 'HRV_SDNN'):
    for j in ('ax_ss', 'ax_so', 'ax_ec', 'ax_pd'):
        m1, m2 = ax_physio[i], behav_simi[j]
        r = spearmanr(m1, m2)
        p = permutation_corr(m1, m2, method='spearman', iter=10000)
        tp = 'Anxiety ' + i +  ' & ' + j
        ax_physio_scale[tp] = r
        if p <= 0.05:
            print('Anxiety ' + i +  ' & ' + j + ' correlation=' + str(r) + ' p=' + str(p))
        else:
            pass

ax_physio_scale

In [ ]:
fe_physio_scale = {}
for i in ('HR', 'EDR', 'HRV_MeanNN', 'HRV_SDNN'):
    for j in ('fe_ss', 'fe_so', 'fe_ec', 'fe_pd'):
        m1, m2 = fe_physio[i], behav_simi[j]
        r = spearmanr(m1, m2)
        p = permutation_corr(m1, m2, method='spearman', iter=10000)
        tp = 'Fear ' + i +  ' & ' + j
        fe_physio_scale[tp] = r
        if p <= 0.05:
            print('Fear ' + i +  ' & ' + j + ' correlation=' + str(r) + ' p=' + str(p))
        else:
            pass

fe_physio_scale

In [ ]:
hl_physio_scale = {}
for i in ('HR', 'EDR', 'HRV_MeanNN', 'HRV_SDNN'):
    for j in ('hl_ss', 'hl_so', 'hl_ec', 'hl_pd'):
        m1, m2 = hl_physio[i], behav_simi[j]
        r = spearmanr(m1, m2)
        p = permutation_corr(m1, m2, method='spearman', iter=10000)
        tp = 'Helpless ' + i +  ' & ' + j
        hl_physio_scale[tp] = r
        if p <= 0.05:
            print('Helpless ' + i +  ' & ' + j + ' correlation=' + str(r) + ' p=' + str(p))
        else:
            pass

hl_physio_scale

In [ ]:
ha_physio_scale = {}
for i in ('HR', 'EDR', 'HRV_MeanNN', 'HRV_SDNN'):
    for j in ('ha_ss', 'ha_so', 'ha_fs'):
        m1, m2 = ha_physio[i], behav_simi[j]
        r = spearmanr(m1, m2)
        p = permutation_corr(m1, m2, method='spearman', iter=10000)
        tp = 'Happy ' + i +  ' & ' + j
        ha_physio_scale[tp] = r
        if p <= 0.05:
            print('Happy ' + i +  ' & ' + j + ' correlation=' + str(r) + ' p=' + str(p))
        else:
            pass

ha_physio_scale

In [ ]:
physio_scale= pd.concat([ag_physio_scale, ax_physio_scale, fe_physio_scale, hl_physio_scale, ha_physio_scale], axis=0)
physio_scale.to_csv(os.path.join(results_dir, 'physio_scale.csv'))